In [ ]:
!curl localhost:9200

In [ ]:
import requests

In [ ]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
print(documents)

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client1 = Elasticsearch('http://localhost:9200')

In [ ]:
es_client1.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-3"

es_client1.indices.create(index=index_name, body=index_settings)

In [ ]:
documents[0]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client1.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query, course='data-engineering-zoomcamp', size=3):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client1.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs, response['hits']['max_score']

In [ ]:
query = "How do I execute a command in a running docker container?"
result_docs, max_score = elastic_search(query)
print(f"Result: {max_score:.2f}")

In [ ]:
query = "How do I execute a command in a running docker container?"
result_docs, _ = elastic_search(query, course='machine-learning-zoomcamp')
third_question = result_docs[2]['question'] if len(result_docs) > 2 else None
print(f"Result: {third_question}")

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join(context_template.format(**doc) for doc in result_docs)

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)

In [ ]:
prompt_length = len(prompt)
print(f"Result: {prompt_length}")

In [ ]:
from transformers import BertTokenizer

def count_tokens(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [ ]:
token_count = count_tokens(prompt)
print(f"Tokens Result: {token_count}")